In [3]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
import os
from pathlib import Path
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
from tifffile import imwrite

%matplotlib widget
%load_ext autoreload
%autoreload 2


/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


2023-05-22 15:30:38.425156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 15:30:38.590600: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/home6/svstaalduine/.local/lib/python3.9/site-packages/cv2/../../lib64:/sw/arch/Centos8/EB_production/2021/software/ZeroMQ/4.3.4-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/util-linux/2.36-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/libsodium/1.0.18-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/OpenPGM/5.2.122-GCCcore-10.3.0/lib:/sw

# Welcome to the kymograph analysis pipeline module!

Use this notebook to load kymographs and extract transport flows from them. The first step is to supply some file parameters to make sure that the measurements are connected to real metrics and that the segmentation method is correct.

Important to note: 
Seg-Tresh is the limit of what percentage of the segmented image should be hypha. 

In [2]:
### Please supply a folder with kymographs. You can filter it based on the title. 
### Doing that makes it easier to set image parameters for each kymo group.
### Spatial resolution should be in micrometers

img_address = 'C:/GitHub_Projects/AMOLF_Data/kymo_requests/Input/*.tif'
out_address = Path('C:/GitHub_Projects/AMOLF_Data/kymo_requests/Output/')
    
# space_resolution is in micrometers per pixel,
# fps is in frames per second
space_resolution = 4*1.725 / 50
fps = 20
display_kymos=True

address_list = [Path(address) for address in glob(img_address)]
kymo_stack  = [imageio.imread(address) for address in address_list]
kymo_edge_list = [Kymo_edge_analysis(kymo=kymo) for kymo in kymo_stack]

for i, kymo_anal in enumerate(tqdm(kymo_edge_list)):
    title = address_list[i].name.split('.')[-2]
    
    imshow_extent = [0, space_resolution * kymo_anal.kymo.shape[1],
                1/fps * kymo_anal.kymo.shape[0], 0]
    kymo_anal.space_pixel_size = space_resolution
    kymo_anal.time_pixel_size = 1/fps    
    forw_thresh, back_thresh = kymo_anal.fourier_kymo(1, test_plots=False)
    speeds, times = kymo_anal.test_GST(15, w_start=3, C_thresh=0.92, C_thresh_falloff = 0.01, blur_size = 3, preblur=True, speed_thresh=50, plots=False)
    x_array = np.array(times[0])
    y_mean_back = np.nanmean(speeds[0][0], axis=1)
    y_std_back = np.nanstd(speeds[0][0], axis=1)
    y_mean_forw = np.nanmean(speeds[0][1], axis=1)
    y_std_forw = np.nanstd(speeds[0][1], axis=1)
    
    fig, ax = plt.subplots(figsize=(9, 9))
    
#     ax[0][0].imshow(forw_thresh[0], extent=imshow_extent, aspect='auto')
#     ax[0][0].set_xlabel("Space ($\mu m$)")
#     ax[0][0].set_ylabel("Time (s)")
#     ax[0][0].set_title("Forward kymo")
    
#     ax[0][1].imshow(back_thresh[0], extent=imshow_extent, aspect='auto')
#     ax[0][1].set_xlabel("Space ($\mu m$)")
#     ax[0][1].set_ylabel("Time (s)")    
#     ax[0][1].set_title("Backward kymo")

#     ax[1][0].imshow(kymo_anal.kymo, extent=imshow_extent, aspect='auto')
#     ax[1][0].set_xlabel("Space ($\mu m$)")
#     ax[1][0].set_ylabel("Time (s)")
#     ax[1][0].set_title("Original kymo")

    ax.errorbar(x_array, y_mean_forw, y_std_forw, alpha=0.5, errorevery=20)
    ax.errorbar(x_array, y_mean_back, y_std_back, alpha=0.5, errorevery=20)
    ax.plot(x_array, y_mean_forw, c='blue', label='Forward mean speed')
    ax.plot(x_array, y_mean_back, c='orange', label='Backward mean speed')
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Speed ($\mu m / s$)")
    ax.legend()
    ax.grid(True)
    ax.set_title(f"{title} Mean speeds with stds")
    plt.savefig(out_address.joinpath(title))
    
    csv_dataframe = pd.DataFrame(
        {
            'y_mean_forw': y_mean_forw,
            'y_mean_back': y_mean_back,
            'y_std_forw' : y_std_forw,
            'y_std_back' : y_std_back,
        }
    )
    csv_dataframe.to_csv(out_address.joinpath(title + '.csv'))
    spds_tiff = np.array([
        speeds[0][0],
        speeds[0][1],
    ], dtype=float)
    imwrite(out_address.joinpath(title+'_spd_fields.tiff'), spds_tiff, photometric='minisblack')

    
if display_kymos:
    for kymo in kymo_stack:
        imshow_extent = [0, space_resolution * kymo.shape[1],
                1/fps * kymo.shape[0], 0]
        fig, ax = plt.subplots(figsize=(8,8))
        ax.imshow(kymo, extent=imshow_extent, aspect='auto')
        ax.set_xlabel("Space ($\mu m$)")
        ax.set_ylabel("Time (s)")
        
        

csv_dataframe
        

100%|██████████| 7/7 [04:32<00:00, 38.95s/it]